In [38]:
import sys
import os
sys.path.insert(0, '/home/jindal/notebooks/jindal/NER')
import fastText
import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
import pickle, threading
from keras.utils import to_categorical
import linecache
from keras.callbacks import Callback

epochs = 50
trainable=True

In [7]:

sys.version_info[0]

3

In [43]:
class WeightsSaver(Callback):
    def __init__(self, model, N):
        self.model = model
        self.N = N
        self.batch = 0

    def on_batch_end(self, batch, logs={}):
        if self.batch % self.N == 0:
            name = 'german_lm.h5'
#             print("model saved %s" %self.batch)
            self.model.save_weights(name)
        self.batch += 1

In [8]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()

In [9]:
def createMatrices(sentences, word2Idx, case2Idx, char2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    
        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []    
        caseIndices = []
        charIndices = []
#         labelIndices = []
        
        for word,char in sentence:  
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]                 
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
            charIdx = []
            for x in char:
                charIdx.append(char2Idx[x])
            #Get the label and map to int            
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
#             labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices,caseIndices, charIndices]) 
        
    return dataset


def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[2]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
    return Sentences

In [ ]:
def doc_load(filename):
    f = open(filename,'rb')
    text = f.read().decode()
    f.close()
    return text

def save_doc(lines, filename):
    data = '\n'.join(lines)
    f = open(filename,'wb')
    f.write(data.encode())
    f.close()


In [ ]:
text = doc_load('/home/jindal/notebooks/wikipedia/de-wiki_tokenized.txt')

In [ ]:
print(text[:200])

In [10]:
window_size =51

In [53]:
def generate_sequences(out_file, in_file ) -> None:
#     x = 0
    output = open(out_file,'wb')
    with open(in_file,'rb') as f:
        for line in f:
#             if x==1000: #To generate toy dataset
#                 break
#             x+=1
            text = line.decode().split()
            for i in range(len(text)):
                if i+1 >= window_size:
                    temp = text[i-window_size+1:i+1]
                else:
                    temp = ['0' for i in range(window_size - (i +1))] + text[:i+1]
                string = ' '.join(temp)+' \n'
#                 if len(string.split())!=51: 
#                     print(len(string.split()))
#                     print(text)
                output.write(string.encode())
    output.close()

In [54]:
generate_sequences('wiki_dataset_sequences','/home/jindal/notebooks/wikipedia/de-wiki_tokenized.txt')

In [16]:
# assert
with open('toy_dataset_sequences_shuffled') as f:
    for line in f:
        assert len(line.split()) == 51

In [17]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [83]:
print(caseEmbeddings.shape)

(8, 8)


In [18]:
characters={}

with open ('/home/jindal/notebooks/jindal/NER/language_model/toy_dataset_sequences_shuffled') as f:
    for line in f:
        for word in line:
            word =str(word)
            for char in word:
#                 print(char)
                characters[char]=True

In [19]:
print(characters)

{'n': True, 'l': True, 'c': True, '‘': True, 'µ': True, ',': True, 'V': True, '“': True, '∧': True, '↔': True, '€': True, '‚': True, 'g': True, 'Δ': True, 'h': True, '„': True, 'd': True, 'μ': True, '"': True, '&': True, 'a': True, '~': True, 'Ö': True, '\n': True, 'S': True, 'í': True, 'f': True, '→': True, 'P': True, '’': True, 'K': True, '¬': True, 'ô': True, 'é': True, 'E': True, 't': True, 'R': True, 'ä': True, 'j': True, '∪': True, 'Ω': True, '…': True, 'N': True, '/': True, 'Z': True, '∨': True, '=': True, 'D': True, '%': True, 'W': True, 'b': True, '†': True, ';': True, 'k': True, 'r': True, 'Y': True, '?': True, 'q': True, ')': True, 'ί': True, 'ß': True, ':': True, 'z': True, 'A': True, '-': True, 'τ': True, 'F': True, 'v': True, 'ö': True, 'X': True, 'C': True, '°': True, 'x': True, 'Q': True, '0': True, 'm': True, '_': True, 'Ä': True, 'H': True, 'B': True, 'U': True, '(': True, 'y': True, 'J': True, 'L': True, 'T': True, '–': True, 'u': True, 'p': True, '*': True, 'Ü': Tru

In [20]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
# print(char2Idx)

In [21]:
toy_vocab ={}

with open('toy_dataset_sequences') as f:
    for line in f:
#         print(line)
        line = line.split()
#         print(len(line))
        for word in line:
#             word = word
#             print(word)
            toy_vocab[word]=True

In [22]:
print(toy_vocab)

{'Weitere': True, 'BIP': True, 'Zukünftiges': True, 'finanzieren': True, 'autobiographische': True, 'Wissensbeständen': True, 'Verwendung': True, 'Hydroxids': True, 'Kasernen': True, 'anzuschließen': True, 'klein': True, 'Tod': True, 'neuer': True, 'Paparazzo': True, 'vermietet': True, 'Verurteilten': True, 'Berlinale': True, 'deutscher': True, '‚': True, 'Teleobjektiv': True, 'Wahrheitstafel': True, 'gesamte': True, 'MacGuffin': True, 'h': True, 'entschloss': True, 'kursiver': True, 'Erlös': True, 'Zugang': True, 'schwarzen': True, 'all': True, 'trug': True, 'Antrag': True, 'metaphysische': True, 'aufzustellen': True, 'handelt': True, 'Bindewort': True, 'Firmen': True, 'Vorgänger': True, 'indem': True, 'Vereinfachung': True, 'Doch': True, 'zustande': True, 'Temperaturen': True, 'Filmgeschäft': True, 'entspräche': True, 'U': True, 'ausgezeichnet': True, 'kein': True, 'K': True, 'Bereich': True, 'Filmwirtschaft': True, 'ihre': True, 'bereits': True, 'Als': True, 'bewahrt': True, 'zukünf

In [23]:
line = linecache.getline('/home/jindal/notebooks/jindal/NER/language_model/toy_dataset_sequences', 17)

print(line)

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 Alan Smithee steht als Pseudonym für einen fiktiven Regisseur , der Filme verantwortet , bei 



In [24]:
word2Idx={}
for word in toy_vocab.keys():
    word2Idx[word] = len(word2Idx)

In [25]:
char2Idx['0']


74

In [26]:
print(word2Idx)

{'Weitere': 0, 'wegen': 2436, 'Zukünftiges': 2, 'klein': 10, 'autobiographische': 4, 'Wissensbeständen': 5, 'Verwendung': 6, 'Hydroxids': 7, 'Kasernen': 8, 'nichtklassisch': 4055, 'hierbei': 3741, 'Tod': 11, 'Auftrag': 2709, 'Paparazzo': 13, 'lang': 2041, 'sondern': 3819, 'vermietet': 14, 'weniger': 3248, 'Berlinale': 16, 'Lanthan': 117, '‚': 18, 'Teleobjektiv': 19, 'Wahrheitstafel': 20, 'gesamte': 21, 'MacGuffin': 22, 'ambitionierter': 69, 'entschloss': 24, 'kursiver': 25, 'Erlös': 26, 'Zugang': 27, 'schwarzen': 28, 'all': 29, 'trug': 30, 'Antrag': 31, 'metaphysische': 32, 'Erziehungssystem': 845, 'handelt': 34, 'Bindewort': 35, 'Firmen': 36, 'Vorgänger': 37, 'indem': 38, 'Vereinfachung': 39, 'Doch': 40, 'zustande': 41, 'Temperaturen': 42, 'ISO': 4060, 'Filmgeschäft': 43, 'entspräche': 44, 'U': 45, 'Lasky': 2570, 'kein': 47, 'assistierte': 282, 'Bereich': 49, 'Sam': 292, 'ihre': 51, 'Verflechtung': 304, 'Besitzern': 316, 'Ressentiments': 1066, 'zukünftige': 55, 'Verknüpfung': 57, 'vor

In [27]:
line_number=0
# lock = threading.Lock()

In [28]:
def my_generator(file :"input training file", batch_size):
    
    global line_number, lock

    while True:
        word_embeddings = []
        case_embeddings = []
        char_embeddings = []
#     batch_features = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims)) # initializing features with zeros
        output_labels = []
        # print(len(features))
        for i in range(batch_size):
            
#             lock.acquire()
            index = line_number%20869 +1
#             print(index)
            line_number+=1
#             lock.release()
#             index = random.choice(len(features), 1)[0]
#             print(index)
            line = linecache.getline(file, index)
            line = line.split()
#             print(line)
            temp_casing = []
            temp_char=[]
            temp_word=[]
#             print(line)
            if len(line)!=51:
                print(index)
                continue
            for word in line[:-1]:
#                 print(word)
                casing =getCasing(word, case2Idx)
#                 print(casing)
                temp_casing.append(casing)
                temp_char2=[]
                for char in word:
                    temp_char2.append(char2Idx[char])
                temp_char2 = np.array(temp_char2)
#                 print(temp_char2)
#                 temp_char2 = pad_sequences(temp_char2, 52, padding='post')
#                 temp_char.append(pad_sequences(temp_char2, 52, padding='post'))
                temp_char.append(temp_char2)
                word_vector = ft.get_word_vector(word.lower())
                temp_word.append(word_vector)
            temp_char = pad_sequences(temp_char, 52)
#             print(temp_word)
#             print(len(temp_word))
#             print(temp_casing)
#             print(temp_char)
#             print(len(temp_char))
#             print(" **************** ")
            word_embeddings.append(temp_word)
            case_embeddings.append(temp_casing)
            char_embeddings.append(temp_char)
            output_labels.append(word2Idx[line[-1]])
#             batch_features[i] = process_features(line, window_size-1, nb_embedding_dims)
            # print(batch_features[i])
            # print(batch_features[i].shape)
#             batch_labels[i] = labels[index]
        yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], np.array(output_labels))

In [ ]:
for inp, output in my_generator('toy_dataset_sequences_shuffled',10):
    continue

In [30]:
vocab_size = len(word2Idx.keys())+1
print(vocab_size)

4851


In [31]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [68]:
words_input = Input(shape=(None, 300), dtype='float32',name='words_input')
# words = Embedding(input_dim =50, output_dim=300, trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words_input, char])
output = Bidirectional(LSTM(200, return_sequences=False, dropout=0.50, recurrent_dropout=0.5))(output)
# output = TimeDistributed(Dense(vocab_size))(output)
# crf = CRF(len(label2Idx))
# output = crf(output)
# output = Flatten()(output)
output = Dense(vocab_size, activation='softmax')(output)
# output = Flatten()(output)
# output = Dense(vocab_size, activation='softmax')(output)
# model.add(Dense(vocab_size, activation='softmax'))
model = Model(inputs=[words_input,casing_input, character_input], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
model.summary()
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 3300        char_input[0][0]                 
__________________________________________________________________________________________________
dropout1 (Dropout)              (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, None, 52, 30) 2730        dropout1[0][0]                   
__________________________________________________________________________________________________
maxpool (T

In [33]:

20700//32

646

In [44]:
line_number=0
try:
    model.fit_generator(my_generator('toy_dataset_sequences_shuffled', 32), epochs=1, steps_per_epoch=20869//32, callbacks=[WeightsSaver(model, 200)])
except Exception as e:
    model.save('german_lm.h5')
model.save('german_lm.h5')

Epoch 1/1
652/652 [==============================] - 145s 222ms/step - loss: 3.9615 - acc: 0.1831


In [72]:
words_input = Input(shape=(None, 300), dtype='float32',name='words_input')
# words = Embedding(input_dim =50, output_dim=300, trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words_input, char])
output = Bidirectional(LSTM(200, return_sequences=False, dropout=0.50, recurrent_dropout=0.5))(output)
# output = TimeDistributed(Dense(vocab_size))(output)
# crf = CRF(len(label2Idx))
# output = crf(output)
# output = Flatten()(output)
output = Dense(vocab_size, activation='softmax')(output)
# output = Flatten()(output)
# output = Dense(vocab_size, activation='softmax')(output)
# model.add(Dense(vocab_size, activation='softmax'))
model_new = Model(inputs=[words_input,casing_input, character_input], outputs=[output])
# model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
# model.summary()

In [73]:
model_new.load_weights('german_lm.h5')

# Train on Twitter 